## Experiments on Instrument Detection Models

11/30/2020

Evaluation Metrics: F1 score, Expected Calibration Error (ECE)

In [1]:
# set up autoreload 
%load_ext autoreload
%autoreload 2

### define data paths

In [2]:
# define paths to datamodules 
dm_paths = {
    'audio': 
    '/home/hugo/CHONK/data/mdb-hop-0.25-chunk-1-AUGMENTED/splits-85-15-REMAPPED', 
    
    'embeddings-512': 
    '/home/hugo/CHONK/data/mdb-hop-0.25-chunk-1-AUGMENTED-EMBEDDINGS-512/splits-85-15-REMAPPED', 

    'embeddings-6144':
    '/home/hugo/CHONK/data/mdb-hop-0.25-chunk-1-AUGMENTED-EMBEDDINGS-6144/splits-85-15-REMAPPED'}


### define a run_trial function

In [3]:
%%capture
from instrument_recognition.task import InstrumentDetectionTask, train_instrument_detection_model
from instrument_recognition.models.zoo import load_model
from instrument_recognition.datasets import load_datamodule
import os
import yaml

def run_trial(exp_dict, test_only=False):

    # load the datamodule
    print(f'loading datamodule...')
    dm = load_datamodule(path_to_data=exp_dict['path_to_data'], 
                         batch_size=exp_dict['batch_size'], 
                         num_workers=exp_dict['num_workers'],
                         use_embeddings=exp_dict['use_embeddings'])
    
    # get classlist and number of classes
    classlist = dm.get_classes()
    num_output_units = len(classlist)
    print(f'classlist is: {classlist}')

    # load model
    print(f'loading model...')
    model = load_model(model_name=exp_dict['model_name'], 
                       output_units=num_output_units, 
                       dropout=exp_dict['dropout'])
    
    # build task
    print(f'building task...')
    task = InstrumentDetectionTask(model, dm, 
                            max_epochs=exp_dict['max_epochs'],
                            learning_rate=exp_dict['learning_rate'], 
                            weighted_cross_entropy=exp_dict['weighted_cross_entropy'], 
                            mixup=exp_dict['mixup'],
                            mixup_alpha=exp_dict['mixup_alpha'], 
                            log_epoch_metrics=True)
    
    # run train fn and get back test results
    print(f'running task')
    result = train_instrument_detection_model(task, 
                                    name=exp_dict['name'], 
                                    version=exp_dict['version'], 
                                    gpuid=exp_dict['gpuid'], 
                                    max_epochs=exp_dict['max_epochs'],
                                    random_seed=exp_dict['random_seed'], 
                                    log_dir=exp_dict['log_dir'],
                                    test_only=test_only,
                                    **exp_dict['trainer_kwargs'])

    # save exp_dict to yaml file for easy reloading
    with open(os.path.join(exp['log_dir'], 'exp_dict.yaml'), 'w') as f:
        yaml.dump(exp_dict, f)
    
    result = result[0]

    return result

### define fixed hyperparameters

we won't change these (except for the batch size in raw audio models)

In [4]:
import datetime
import os
now = datetime.datetime.now().strftime("%d.%m.%Y")
log_dir = os.path.join('logs', f'experiment-{now}')

# define experiment
FIXED = dict(batch_size=128*3, num_workers=20, learning_rate=3e-4,
             weighted_cross_entropy=True, dropout=0.5, random_seed=20, 
             max_epochs=80, log_dir=log_dir,  version=0,  gpuid=-1,
             trainer_kwargs={}) 
    
def make_trial(trial_dict):
    """ creates a dict with fixed hyperparams
    and then adds trial_dict on top
    """
    exp_dict = dict(FIXED) 
    exp_dict.update(trial_dict)
    return exp_dict


### define all trials

In [5]:
trials = [
    dict(name='openl3-mlp-ERM',
    path_to_data=dm_paths['embeddings-6144'],
    use_embeddings= True, 
    model_name='mlp-6144',
    mixup=False,
    mixup_alpha=0), 
    
    dict(name='finetuned-mlp-ERM', 
     path_to_data=dm_paths['audio'], 
     batch_size=32*3,
     use_embeddings=False, 
     model_name='openl3mlp-6144', 
     mixup=False, 
     mixup_alpha=0,
     trainer_kwargs=dict(accumulate_grad_batches=8)),
    
    dict(name='baseline-mlp-ERM', 
     path_to_data=dm_paths['audio'], 
     batch_size=32*3,
     use_embeddings=False, 
     model_name='baseline-6144', 
     mixup=False, 
     mixup_alpha=0,
     trainer_kwargs=dict(accumulate_grad_batches=8)),
    
    dict(name='openl3-mlp-MIXUP-alpha=0.2',
    path_to_data=dm_paths['embeddings-6144'],
    use_embeddings= True, 
    model_name='mlp-6144',
    mixup=True,
    mixup_alpha=0.2),
    
    dict(name='baseline-mlp-MIXUP-alpha=0.2', 
     path_to_data=dm_paths['audio'], 
     batch_size=32*3,
     use_embeddings=False, 
     model_name='baseline-6144', 
     mixup=True, 
     mixup_alpha=0.2,
     trainer_kwargs=dict(accumulate_grad_batches=8)),
    
    dict(name='finetuned-mlp-MIXUP-alpha=0.2', 
     path_to_data=dm_paths['audio'], 
     batch_size=32*3,
     use_embeddings=False, 
     model_name='openl3mlp-6144', 
     mixup=True, 
     mixup_alpha=0.2,
     trainer_kwargs=dict(accumulate_grad_batches=8)),
    
    # MODELS WITH MIXUP (ALPHA = 0.4)
    dict(name='openl3-mlp-MIXUP-alpha=0.4',
        path_to_data=dm_paths['embeddings-6144'],
        use_embeddings= True, 
        model_name='mlp-6144',
        mixup=True,
        mixup_alpha=0.4), 

    dict(name='baseline-mlp-MIXUP-alpha=0.4', 
     path_to_data=dm_paths['audio'], 
     batch_size=32*3,
     use_embeddings=False, 
     model_name='baseline-6144', 
     mixup=True, 
     mixup_alpha=0.4,
     trainer_kwargs=dict(accumulate_grad_batches=8)),
    
    dict(name='finetuned-mlp-MIXUP-alpha=0.4', 
     path_to_data=dm_paths['audio'], 
     batch_size=32*3,
     use_embeddings=False, 
     model_name='openl3mlp-6144', 
     mixup=True, 
     mixup_alpha=0.4,
     trainer_kwargs=dict(accumulate_grad_batches=8))
]
trials = {t['name']: make_trial(t) for t in trials}

## train models!! :) 

In [6]:
# run me too see what trials I can run
for k in trials:
    print(k)

openl3-mlp-ERM
finetuned-mlp-ERM
baseline-mlp-ERM
openl3-mlp-MIXUP-alpha=0.2
baseline-mlp-MIXUP-alpha=0.2
finetuned-mlp-MIXUP-alpha=0.2
openl3-mlp-MIXUP-alpha=0.4
baseline-mlp-MIXUP-alpha=0.4
finetuned-mlp-MIXUP-alpha=0.4


In [ ]:
%%capture cap
run_trial(trials['openl3-mlp-ERM'])

In [ ]:
%%capture cap
run_trial(trials['openl3-mlp-MIXUP-alpha=0.2'])

In [ ]:
%%capture cap
run_trial(trials['openl3-mlp-MIXUP-alpha=0.4'])

In [ ]:
run_trial(trials['baseline-mlp-ERM'])

loading datamodule...
medley db path: /home/hugo/data/medleydb


/home/hugo/lab/venv/lib/python3.6/site-packages/medleydb/__init__.py:65: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.

/home/hugo/lab/venv/lib/python3.6/site-packages/medleydb/__init__.py:73: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



train entries: 288692
val entries: 56020
classlist is: ['acoustic guitar', 'auxiliary percussion', 'brass section', 'cello', 'clean electric guitar', 'distorted electric guitar', 'double bass', 'drum set', 'electric bass', 'female singer', 'male singer', 'oboe', 'piano', 'synthesizer', 'tack piano', 'trumpet', 'vibraphone', 'viola', 'violin', 'vocalists']
loading model...


INFO:lightning:GPU available: True, used: True
INFO:lightning:TPU available: False, using: 0 TPU cores
INFO:lightning:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
INFO:lightning:Using native 16bit precision.


building task...
running task


/home/hugo/lab/venv/lib/python3.6/site-packages/torch/nn/parallel/data_parallel.py:30: UserWarning:


    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.

INFO:lightning:
   | Name                                      | Type            | Params
-------------------------------------------------------------------------------
0  | model                                     | OpenL3MLP       | 12 M  
1  | model.openl3                              | OpenL3Embedding | 9 M   
2  | model.openl3.filters                      | Melspectrogram  | 4 M   
3  | model.openl3.filters.conv1d_real          | Conv1d          | 2 M   
4  | model.openl3.filters.conv1d_imag          | Conv1d          | 2 M   
5  | model.openl3.filters.freq2mel             | Linear          | 262 K 
6  | model.o

Test tube created git tag: tt_baseline-mlp-ERM_v0


/home/hugo/lab/venv/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:64: UserWarning:

Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.



INFO:lightning:Epoch 0: loss/val reached 1.31725 (best 1.31725), saving model to /home/hugo/lab/mono_music_sed/instrument_recognition/notebooks/logs/experiment-03.12.2020/baseline-mlp-ERM/version_0/checkpoints/epoch=00-loss_val=1.32.ckpt as top 3


In [ ]:
run_trial(trials['baseline-mlp-MIXUP-alpha=0.2'])

In [ ]:
run_trial(trials['baseline-mlp-MIXUP-alpha=0.4'])

In [ ]:
run_trial(trials['finetuned-mlp-ERM'])

In [ ]:
run_trial(trials['finetuned-mlp-MIXUP-alpha=0.2'])

In [ ]:
run_trial(trials['finetuned-mlp-MIXUP-alpha=0.4'])

## test models :)

In [ ]:
import traceback 
import pandas as pd

scoreboard = []
for exp in exp_dicts:
    print(100*'-', '\n', 100*'-')
    try:
        if not exp['done']:
            results = run_trial(exp)
            exp['done'] = True
        else:
            print(f'skipping {exp["name"]}')
    except:
        print(f'exp: {exp["name"]} failed.')
        traceback.print_exc(limit=None, file=None, chain=True)
        results = {'error': True}
        
    score = dict(exp)
    score.update(results)
    scoreboard.append(score)
    
scoreboard_df = pd.DataFrame(scoreboard)

In [ ]:
%store

In [ ]:
scoreboard_df.to_csv(os.path.join(log_dir, 'scoreboard.csv'))
with open(os.path.join(log_dir, 'output.txt'),  'w') as f:
    f.write(cap.stdout)